# 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么需要定义模型中的所有参数，当然这样可以非常方便地定义每个变量的初始化方式。但是对于复杂的模型，这并不容易，而且推荐使用 Sequential 和 Module 来定义模型，所以这个时候就需要知道如何来自定义初始化方式。

## 1. 使用 NumPy 来初始化
因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [3]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [4]:
print(w1)

Parameter containing:
tensor([[ 0.1249, -0.0230,  0.0798,  ..., -0.0423,  0.1062, -0.1041],
        [-0.1512, -0.0862,  0.1001,  ...,  0.0706, -0.0672,  0.1552],
        [ 0.0622, -0.0738, -0.0148,  ..., -0.0022,  0.1169, -0.1714],
        ...,
        [ 0.1528, -0.1007,  0.1237,  ..., -0.0426,  0.0390,  0.0906],
        [ 0.0149,  0.1707,  0.0566,  ..., -0.1777,  0.1292, -0.1740],
        [-0.1424,  0.1188, -0.0467,  ...,  0.0724, -0.0363,  0.1626]],
       requires_grad=True)


注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 `.data`属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 `torch.randn`，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [5]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [6]:
print(net1[0].weight)

Parameter containing:
tensor([[3.0862, 3.4411, 3.7954,  ..., 4.8664, 4.0997, 3.6628],
        [4.9739, 3.3945, 4.3667,  ..., 4.9704, 4.6954, 4.9397],
        [3.3513, 3.4033, 4.4537,  ..., 3.6112, 3.8753, 3.9057],
        ...,
        [3.9480, 3.2882, 3.8318,  ..., 4.2120, 4.3537, 3.8374],
        [4.7273, 3.6481, 3.8571,  ..., 4.1158, 4.0978, 3.6291],
        [3.3689, 3.9035, 3.6940,  ..., 3.8595, 3.9986, 4.0478]],
       dtype=torch.float64, requires_grad=True)


可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [7]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

一种非常流行的初始化方式叫 Xavier，方法来源于 2010 年的一篇论文 [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)，其通过数学的推导，证明了这种初始化方式可以使得每一层的输出方差是尽可能相等。这种初始化的公式为：

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

其中 $n_j$ 和 $n_{j+1}$ 表示该层的输入和输出数目，所以请尝试实现以下这种初始化方式

对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [8]:
class Simp_Net(nn.Module):
    def __init__(self):
        super(Simp_Net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(40, 30) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [9]:
net2 = Simp_Net()

In [10]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


In [11]:
# 访问 modules
for i in net2.modules():
    print(i)

Simp_Net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Linear(in_features=30, out_features=40, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Linear(in_features=40, out_features=50, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)
Linear(in_features=50, out_features=10, bias=True)
ReLU()


通过上面的例子，看到区别了吗?

children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [12]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

这上面实现了和 Sequential 相同的初始化，同样非常简便

## 2. `torch.nn.init`
因为 PyTorch 灵活的特性，可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 `torch.nn.init`，其操作层面仍然在 Tensor 上，下面我们举例说明

In [13]:
from torch.nn import init

In [14]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.4076, -0.7389,  0.0897,  ...,  0.0583,  0.5968, -0.6215],
        [ 0.8528,  0.7439, -0.1739,  ...,  0.5936, -0.7346, -0.3665],
        [ 0.6312, -0.1565, -0.3483,  ...,  0.3290,  0.0294,  0.5446],
        ...,
        [-0.2233,  0.2200,  0.1605,  ...,  0.4023,  0.6479, -0.6087],
        [ 0.6451, -0.1437, -0.1425,  ...,  0.5666,  0.2419,  0.4368],
        [ 0.6162,  0.4614, -0.3018,  ..., -0.0792, -0.9643, -0.8497]],
       dtype=torch.float64, requires_grad=True)


In [15]:
init.xavier_uniform_(net1[0].weight) # 这就是上面我们讲过的 Xavier 初始化方法，PyTorch 直接内置了其实现

Parameter containing:
tensor([[ 0.0990, -0.0568, -0.0579,  ...,  0.0061, -0.1130,  0.1382],
        [ 0.2898,  0.1651, -0.2005,  ..., -0.2800, -0.0734,  0.0893],
        [-0.1399,  0.2747,  0.1557,  ..., -0.1936,  0.2899, -0.1261],
        ...,
        [ 0.0850,  0.2906,  0.2412,  ..., -0.2687, -0.0496, -0.2268],
        [-0.1026,  0.0337,  0.2273,  ..., -0.1480,  0.1660, -0.1967],
        [-0.2741, -0.1837,  0.0464,  ...,  0.2838, -0.1822, -0.2453]],
       dtype=torch.float64, requires_grad=True)

In [16]:
print(net1[0].weight)

Parameter containing:
tensor([[ 0.0990, -0.0568, -0.0579,  ...,  0.0061, -0.1130,  0.1382],
        [ 0.2898,  0.1651, -0.2005,  ..., -0.2800, -0.0734,  0.0893],
        [-0.1399,  0.2747,  0.1557,  ..., -0.1936,  0.2899, -0.1261],
        ...,
        [ 0.0850,  0.2906,  0.2412,  ..., -0.2687, -0.0496, -0.2268],
        [-0.1026,  0.0337,  0.2273,  ..., -0.1480,  0.1660, -0.1967],
        [-0.2741, -0.1837,  0.0464,  ...,  0.2838, -0.1822, -0.2453]],
       dtype=torch.float64, requires_grad=True)


可以看到参数已经被修改了

`torch.nn.init` 提供了更多的内置初始化方式，避免了重复去实现一些相同的操作。

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 `torch.nn.init` 提供了更多成熟的深度学习相关的初始化方式。
